### Setup Background

In [ ]:
!apt-get update
!apt-get upgrade

In [ ]:
!apt-get install build-essential
!apt-get install cmake git libgtk2.0-dev pkg-config libavcodec-dev libavformat-dev libswscale-dev

In [ ]:
!apt-get install libavcodec-dev libavformat-dev libswscale-d

In [ ]:
!apt-get -y install cmake
!which cmake

!cmake --version

In [ ]:
!apt-get install libopencv-dev

In [ ]:
!git clone https://github.com/AlexeyAB/darknet/

In [ ]:
!apt-get install vim

In [ ]:
%cd darknet

In [ ]:
!ls
!wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
!ls
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile

In [ ]:
!ls
%cd ../
!ls

In [ ]:
!apt install g++-5
!apt install gcc-5

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 10
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 10
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20
!update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!update-alternatives --set cc /usr/bin/gcc
!update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!update-alternatives --set c++ /usr/bin/g++

In [ ]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!apt-get update -qq
!apt-get install cuda -y -qq gcc-5 g++-5 
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++
!apt update
!apt upgrade
!apt install cuda-8.0 -y

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

print("'sup!'")

!/usr/local/cuda/bin/nvcc --version

In [ ]:
%cd darknet
!make

In [ ]:
%cd ..
%cd ..
!cp /mydrive/videos/CCTV.mp4 /content/darknet/data

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()
  
  
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd data
!ls

### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Validation with darknet_video.py
%cd /content/darknet/
!python darknet.py

In [ ]:
# Check Files in Google Drive
!ln -s /content/drive/My\ Drive/ /mydrive

In [ ]:
#Using Pre-trained weights file to validate CCTV.mp4
!ls
%cd /content/darknet
!./darknet detector demo cfg/coco.data cfg/yolov3.cfg yolov3.weights -dont_show /mydrive/videos/CCTV.mp4 -i 0 -out_filename yolov3_CCTV.avi

### Data Copy

In [ ]:
!ls /mydrive/yolov3/

In [ ]:
!cp /mydrive/data.zip /content/darknet/
!unzip /content/darknet/data.zip -d /content/darknet/data/

In [ ]:
!cp /mydrive/yolov3.cfg /content/darknet/cfg/
!cp /mydrive/obj.data /content/darknet/data/
!cp /mydrive/obj.names /content/darknet/data/
!cp /mydrive/train.txt /content/darknet/data/

In [ ]:
!cp /mydrive/yolov3/darknet53.conv.74 /content/darknet/

In [ ]:
!cp /mydrive/yolov3/yolov3-tiny.conv.15 /content/darknet/

In [ ]:
%matplotlib notebook

In [ ]:
%cd /content/darknet
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 416 -height 416 -show

Training

In [ ]:
# Train From Scratch
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3.cfg /content/darknet/darknet53.conv.74 -dont_show

In [ ]:
# Train From Saved File
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3.cfg /mydrive/yolov3/backup/yolov3_last.weights -dont_show

Training - Tiny

In [ ]:
# Train From Scratch
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolo-obj.cfg /content/darknet/yolov3-tiny.conv.15 -dont_show

In [ ]:
# Train From Saved File
%cd /content/darknet/
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolo-obj.cfg /mydrive/yolov3/backup/tiny-yolo-obj_last.weights -dont_show

### Test Video

In [ ]:
!cp /mydrive/tiny-yolo-obj.cfg /content/darknet/cfg/
!cp /mydrive/yolov3/backup/tiny-yolo-obj_last.weights /content/darknet/backup/

In [ ]:
#resizing batch & subdivision
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' tiny-yolo-obj.cfg
!sed -i 's/subdivisions=64/subdivisions=1/' tiny-yolo-obj.cfg
%cd ..

In [ ]:
# Video Validation
%cd /content/darknet
!./darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/tiny-yolo-obj.cfg /mydrive/yolov3/backup/tiny-yolo-obj_last.weights -dont_show /mydrive/test/deercar.mp4 -i 0 -out_filename yolov3_out_deercar_tiny.mp4

In [ ]:
#resizing batch & subdivision
%cd /content/darknet/cfg
!sed -i 's/batch=64/batch=1/' yolov3.cfg
!sed -i 's/subdivisions=64/subdivisions=1/' yolov3.cfg
%cd ..

In [ ]:
#Here's the command for processing video, make sure you have the weights and
#It'll be saved as result.avi
%cd /content/darknet/
!./darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/yolov3.cfg /mydrive/yolov3/backup/yolov3_last.weights -dont_show /mydrive/test/deercar.mp4 -i 0 -out_filename yolov3_out_deercar_v3.mp4

In [ ]:
# Video Validation
%cd ..
%cd ..
%cd /content/darknet
!./darknet detector demo /content/darknet/data/obj.data /content/darknet/cfg/yolov3.cfg /mydrive/yolov3/backup/yolov3_last.weights -dont_show /mydrive/test/deeeer.mp4 -i 0 -out_filename deeeer_test1.avi

In [ ]:
%cd /content/darknet
!python darknet_video.py

Test

In [ ]:
%cd ..
%cd ..
%cd /content/darknet/
!./darknet detector

### training & validation with yolo_tiny

In [ ]:
#Then you can download the video to your machine, just call
#download('name_of_the_video') , and keep throwing !ls's !!
!ls

In [ ]:
download('fiddlevideo1.avi')

In [ ]:
#After you've trained your model you can also run it against an image
!wget https://s9.stc.all.kpcdn.net/share/i/12/10186046/inx960x640.jpg
!./darknet detector test data/obj.data yolov3-tiny-obj.cfg backup/yolov3-tiny-obj_1000.weights inx960x640.jpg -i 0 -thresh 0.15
#Show the image
imShow('predictions.jpg')

In [ ]:
#Okay, so if you're familiar with AlexeyAB's darknet fork you know that we'll
#need 5 things:
#Obj folder (training data)
#Obj.data, Obj.name file
#train.txt file
#Weights and the edited CFG model 
#Go here https://github.com/AlexeyAB/darknet/ to figure out what you need
#So, you'll basically need to edit all that stuff on your machine and prepare
#it to be  uploaded, unless you're a ninja can do that all here, which I can't -_-


# Use %cd <directory> and %cd ../ to navigate the directories to put everything
#in place

In [ ]:
#Upload cfg file (make sure that you're in darknet folder)
#upload()
#and check the results:
#!ls
#Just copy this code and repeat it for as long as you need

In [ ]:
#Then call
#!unzip obj.zip
#And there you have a folder with your data


In [ ]:
#Now, once you've put everything in place, you can run the command, in my case that was:
!./darknet detector train data/obj.data yolov3-tiny-obj.cfg yolov3-tiny.conv.15 -dont_show
#Don't forget the dont_show flag, it's good for your health ;)

In [ ]:
#Once the model's trained you can go to the backup folder
#and call download('weights') to get the trained weights you want, in my case
download('yolov3-tiny-obj_1500.weights')

In [ ]:
#So, there we go! I mean, the hard part is to get everything up and going 
#And you can do pretty much anything once you did that))
#Feel free to correct the errors (when something will stop working in the future)
#And if that was useful, my dear AI wizard, go and do some good and cool stuff with it! =)